In [ ]:
import sys
import subprocess
from collections import Counter
from collections import defaultdict
import scanpy as sc
import pandas as pd
import pickle as pkl
import numpy as np
import scipy
import matplotlib.pyplot as plt
import re
import glob
import os
import sys

# from geosketch import gs
from numpy import cov
import scipy.cluster.hierarchy as spc
import seaborn as sns

sns.set(color_codes=True)
from sklearn.linear_model import LogisticRegression
import sklearn
from pathlib import Path
import requests
import psutil
import random
import threading
import tracemalloc
import itertools
import math
import warnings
import sklearn.metrics as metrics
from collections import Counter
from collections import defaultdict
import scanpy as sc
import pandas as pd
import pickle as pkl
import numpy as np
import scipy
import matplotlib.pyplot as plt
import re
import glob
import os
import sys

# from geosketch import gs
from numpy import cov
import scipy.cluster.hierarchy as spc
import seaborn as sns

sns.set(color_codes=True)
from sklearn.linear_model import LogisticRegression
import sklearn
from pathlib import Path
import requests
import psutil
import random
import threading
import tracemalloc
import itertools
import math
import warnings
import sklearn.metrics as metrics
import numpy as np
from sklearn.metrics import log_loss
import mygene
import gseapy as gp
import mygene
import scipy.sparse as sparse
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn import metrics
import seaborn as sn
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import pymc3 as pm
from scipy.sparse import csr_matrix
from scipy.stats import entropy

# main_probabillistic_training_projection_modules

In [4]:
def load_adatas(adatas_dict, data_merge, data_key_use, QC_normalise):
    """
    General description.

    Parameters:

    Returns:

    """
    if data_merge == True:
        # Read
        gene_intersect = {}  # unused here
        adatas = {}
        for dataset in adatas_dict.keys():
            if "https" in adatas_dict[dataset]:
                print("Loading anndata from web source")
                adatas[dataset] = sc.read(
                    "./temp_adata.h5ad", backup_url=adatas_dict[dataset]
                )
            adatas[dataset] = sc.read(data[dataset])
            adatas[dataset].var_names_make_unique()
            adatas[dataset].obs["dataset_merge"] = dataset
            adatas[dataset].obs["dataset_merge"] = dataset
            gene_intersect[dataset] = list(adatas[dataset].var.index)
        adata = list(adatas.values())[0].concatenate(
            list(adatas.values())[1:], join="inner"
        )
        return adatas, adata
    elif data_merge == False:
        if "https" in adatas_dict[data_key_use]:
            print("Loading anndata from web source")
            adata = sc.read("./temp_adata.h5ad", backup_url=adatas_dict[data_key_use])
        else:
            adata = sc.read(adatas_dict[data_key_use])
    if QC_normalise == True:
        print(
            "option to apply standardisation to data detected, performing basic QC filtering"
        )
        sc.pp.filter_cells(adata, min_genes=200)
        sc.pp.filter_genes(adata, min_cells=3)
        sc.pp.normalize_per_cell(adata, counts_per_cell_after=1e4)
        sc.pp.log1p(adata)

    return adata


# projection module
def reference_projection(
    adata, model, dyn_std, partial_scale, train_x_partition, **kwargs
):
    """
    General description.

    Parameters:

    Returns:

    """

    class adata_temp:
        pass

    from sklearn.preprocessing import StandardScaler

    print("Determining model flavour")
    try:
        model_lr = model["Model"]
        print("Consuming celltypist model")
    except:  # hasattr(model, 'coef_'):
        print("Consuming non-celltypist model")
        model_lr = model
    print(model_lr)
    if train_x_partition == "X":
        print("Matching reference genes in the model")
        k_x = np.isin(list(adata.var.index), list(model_lr.features))
        if k_x.sum() == 0:
            raise ValueError(
                f"🛑 No features overlap with the model. Please provide gene symbols"
            )
        print(f"🧬 {k_x.sum()} features used for prediction")
        # slicing adata
        k_x_idx = np.where(k_x)[0]
        # adata_temp = adata[:,k_x_idx]
        adata_temp.var = adata[:, k_x_idx].var
        adata_temp.X = adata[:, k_x_idx].X
        adata_temp.obs = adata[:, k_x_idx].obs
        lr_idx = (
            pd.DataFrame(model_lr.features, columns=["features"])
            .reset_index()
            .set_index("features")
            .loc[list(adata_temp.var.index)]
            .values
        )
        # adata_arr = adata_temp.X[:,list(lr_idexes['index'])]
        # slice and reorder model
        ni, fs, cf = model_lr.n_features_in_, model_lr.features, model_lr.coef_
        model_lr.n_features_in_ = lr_idx.size
        model_lr.features = np.array(model_lr.features)[lr_idx]
        model_lr.coef_ = np.squeeze(
            model_lr.coef_[:, lr_idx]
        )  # model_lr.coef_[:, lr_idx]
        if partial_scale == True:
            print(
                "scaling input data, default option is to use incremental learning and fit in mini bulks!"
            )
            # Partial scaling alg
            scaler = StandardScaler(with_mean=False)
            n = adata_temp.X.shape[0]  # number of rows
            # set dyn scale packet size
            x_len = len(adata_temp.var)
            y_len = len(adata.obs)
            if y_len < 100000:
                dyn_pack = int(x_len / 10)
                pack_size = dyn_pack
            else:
                # 10 pack for every 100,000
                dyn_pack = int((y_len / 100000) * 10)
                pack_size = int(x_len / dyn_pack)
            batch_size = (
                1000  # pack_size#500  # number of rows in each call to partial_fit
            )
            index = 0  # helper-var
            while index < n:
                partial_size = min(
                    batch_size, n - index
                )  # needed because last loop is possibly incomplete
                partial_x = adata_temp.X[index : index + partial_size]
                scaler.partial_fit(partial_x)
                index += partial_size
            adata_temp.X = scaler.transform(adata_temp.X)
    # model projections
    print("Starting reference projection!")
    if train_x_partition == "X":
        train_x = adata_temp.X
        pred_out = pd.DataFrame(
            model_lr.predict(train_x),
            columns=["predicted"],
            index=list(adata.obs.index),
        )
        proba = pd.DataFrame(
            model_lr.predict_proba(train_x),
            columns=model_lr.classes_,
            index=list(adata.obs.index),
        )
        pred_out = pred_out.join(proba)

    elif train_x_partition in list(adata.obsm.keys()):
        print("{low_dim: this partition modality is still under development!}")
        train_x = adata.obsm[train_x_partition]
        pred_out = pd.DataFrame(
            model_lr.predict(train_x),
            columns=["predicted"],
            index=list(adata.obs.index),
        )
        proba = pd.DataFrame(
            model_lr.predict_proba(train_x),
            columns=model_lr.classes_,
            index=list(adata.obs.index),
        )
        pred_out = pred_out.join(proba)

    else:
        print("{this partition modality is still under development!}")
    ## insert modules for low dim below

    # Simple dynamic confidence calling
    pred_out["confident_calls"] = pred_out["predicted"]
    pred_out.loc[
        pred_out.max(axis=1) < (pred_out.mean(axis=1) + (1 * pred_out.std(axis=1))),
        "confident_calls",
    ] = (
        pred_out.loc[
            pred_out.max(axis=1) < (pred_out.mean(axis=1) + (1 * pred_out.std(axis=1))),
            "confident_calls",
        ].astype(str)
        + "_uncertain"
    )
    # means_ = self.model.scaler.mean_[lr_idx] if self.model.scaler.with_mean else 0
    return (pred_out, train_x, model_lr, adata_temp)


# Modified LR train module, does not work with low-dim by default anymore, please use low-dim adapter
def LR_train(
    adata,
    train_x,
    train_label,
    penalty="elasticnet",
    sparcity=0.2,
    max_iter=200,
    l1_ratio=0.2,
    tune_hyper_params=False,
    n_splits=5,
    n_repeats=3,
    l1_grid=[0.01, 0.2, 0.5, 0.8],
    c_grid=[0.01, 0.2, 0.4, 0.6],
    thread_num=-1,
    **kwargs,
):
    """
    General description.

    Parameters:

    Returns:

    """
    if tune_hyper_params == True:
        train_labels = train_label
        results = tune_lr_model(
            adata,
            train_x_partition=train_x,
            random_state=42,
            train_labels=train_labels,
            n_splits=n_splits,
            n_repeats=n_repeats,
            l1_grid=l1_grid,
            c_grid=c_grid,
        )
        print("hyper_params tuned")
        sparcity = results.best_params_["C"]
        l1_ratio = results.best_params_["l1_ratio"]

    lr = LogisticRegression(
        penalty=penalty, C=sparcity, max_iter=max_iter, n_jobs=thread_num
    )
    if penalty == "l1":
        lr = LogisticRegression(
            penalty=penalty,
            C=sparcity,
            max_iter=max_iter,
            dual=True,
            solver="liblinear",
            multi_class="ovr",
            n_jobs=thread_num,
        )  # one-vs-rest
    if penalty == "elasticnet":
        lr = LogisticRegression(
            penalty=penalty,
            C=sparcity,
            max_iter=max_iter,
            dual=False,
            solver="saga",
            l1_ratio=l1_ratio,
            multi_class="ovr",
            n_jobs=thread_num,
        )
    if train_x == "X":
        subset_train = adata.obs.index
        # Define training parameters
        train_label = adata.obs[train_label].values
        #        predict_label = train_label[subset_predict]
        #        train_label = train_label[subset_train]
        train_x = adata.X  # [adata.obs.index.isin(list(adata.obs[subset_train].index))]
    #        predict_x = adata.X[adata.obs.index.isin(list(adata.obs[subset_predict].index))]
    elif train_x in adata.obsm.keys():
        # Define training parameters
        train_label = adata.obs[train_label].values
        #        predict_label = train_label[subset_predict]
        #         train_label = train_label[subset_train]
        train_x = adata.obsm[train_x]
    #        predict_x = train_x
    #        train_x = train_x[subset_train, :]
    # Define prediction parameters
    #        predict_x = predict_x[subset_predict]
    #        predict_x = pd.DataFrame(predict_x)
    #        predict_x.index = adata.obs[subset_predict].index
    # Train predictive model using user defined partition labels (train_x ,train_label, predict_x)
    model = lr.fit(train_x, train_label)
    model.features = np.array(adata.var.index)
    return model


def tune_lr_model(
    adata,
    train_x_partition="X",
    random_state=42,
    use_bayes_opt=True,
    train_labels=None,
    n_splits=5,
    n_repeats=3,
    l1_grid=[0.1, 0.2, 0.5, 0.8],
    c_grid=[0.1, 0.2, 0.4, 0.6],
    thread_num=-1,
    **kwargs,
):
    """
    General description.

    Parameters:

    Returns:

    """
    import bless as bless
    from sklearn.gaussian_process.kernels import RBF
    from numpy import arange
    from sklearn.model_selection import RepeatedKFold
    from sklearn.datasets import make_classification
    from sklearn.linear_model import LogisticRegression
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import f1_score
    from sklearn.model_selection import GridSearchCV
    from skopt import BayesSearchCV

    # If latent rep is provided, randomly sample data in spatially aware manner for initialisation
    r = np.random.RandomState(random_state)
    if train_x_partition in adata.obsm.keys():
        lvg = bless.bless(
            tune_train_x,
            RBF(length_scale=20),
            lam_final=2,
            qbar=2,
            random_state=r,
            H=10,
            force_cpu=True,
        )
        #     try:
        #         import cupy
        #         lvg_2 = bless(adata.obsm[train_x_partition], RBF(length_scale=10), 10, 10, r, 10, force_cpu=False)
        #     except ImportError:
        #         print("cupy not found, defaulting to numpy")
        adata_tuning = adata[lvg.idx]
        tune_train_x = adata_tuning.obsm[train_x_partition][:]
    else:
        print("no latent representation provided, random sampling instead")
        prop = 0.1
        random_vertices = []
        n_ixs = int(len(adata.obs) * prop)
        random_vertices = random.sample(list(range(len(adata.obs))), k=n_ixs)
        adata_tuning = adata[random_vertices]
        tune_train_x = adata_tuning.X

    if not train_labels == None:
        tune_train_label = adata_tuning.obs[train_labels]
    elif train_labels == None:
        try:
            print(
                "no training labels provided, defaulting to unsuperived leiden clustering, updates will change this to voronoi greedy sampling"
            )
            sc.tl.leiden(adata_tuning)
        except:
            print(
                "no training labels provided, no neighbors, defaulting to unsuperived leiden clustering, updates will change this to voronoi greedy sampling"
            )
            sc.pp.neighbors(adata_hm, n_neighbors=15, n_pcs=50)
            sc.tl.leiden(adata_tuning)
        tune_train_label = adata_tuning.obs["leiden"]
    ## tune regularization for multinomial logistic regression
    print("starting tuning loops")
    X = tune_train_x
    y = tune_train_label
    grid = dict()
    # define model
    cv = RepeatedKFold(
        n_splits=n_splits, n_repeats=n_repeats, random_state=random_state
    )
    # model = LogisticRegression(penalty = penalty, max_iter =  200, dual=False,solver = 'saga', multi_class = 'multinomial',)
    model = LogisticRegression(
        penalty=penalty, C=sparcity, max_iter=100, n_jobs=thread_num
    )
    if penalty == "l1":
        model = LogisticRegression(
            penalty=penalty,
            C=sparcity,
            max_iter=100,
            dual=True,
            solver="liblinear",
            multi_class="multinomial",
            n_jobs=thread_num,
        )  # one-vs-rest
    if penalty == "elasticnet":
        model = LogisticRegression(
            penalty=penalty,
            C=sparcity,
            max_iter=100,
            dual=False,
            solver="saga",
            l1_ratio=l1_ratio,
            multi_class="multinomial",
            n_jobs=thread_num,
        )  # use multinomial class if probabilities are descrete
        grid["l1_ratio"] = l1_grid
    grid["C"] = c_grid

    if use_bayes_opt == True:
        # define search space
        search_space = {
            "C": (np.min(c_grid), np.max(c_grid), "log-uniform"),
            "l1_ratio": (np.min(l1_grid), np.max(l1_grid), "uniform")
            if "elasticnet" in penalty
            else None,
        }
        # define search
        search = BayesSearchCV(
            model,
            search_space,
            scoring="neg_mean_absolute_error",
            cv=cv,
            n_jobs=-thread_num,
        )
        # perform the search
        results = search.fit(X, y)
    else:
        # define search
        search = GridSearchCV(
            model, grid, scoring="neg_mean_absolute_error", cv=cv, n_jobs=thread_num
        )
        # perform the search
        results = search.fit(X, y)
    # summarize
    print("MAE: %.3f" % results.best_score_)
    print("Config: %s" % results.best_params_)
    return results


def prep_training_data(
    adata_temp,
    feat_use,
    batch_key,
    model_key,
    batch_correction=False,
    var_length=7500,
    penalty="elasticnet",
    sparcity=0.2,
    max_iter=200,
    l1_ratio=0.1,
    partial_scale=True,
    train_x_partition="X",
    theta=3,
    tune_hyper_params=False,
    thread_num=-1,
    **kwargs,
):
    """
    General description.

    Parameters:

    Returns:

    """
    model_name = model_key + "_lr_model"
    # scale the input data
    if partial_scale == True:
        print(
            "scaling input data, default option is to use incremental learning and fit in mini bulks!"
        )
        print("performing highly variable gene selection")
        sc.pp.highly_variable_genes(adata_temp, batch_key=batch_key, subset=False)
        adata_temp = subset_top_hvgs(adata_temp, var_length)
        # Partial scaling alg
        # adata_temp.X = (adata_temp.X)
        scaler = StandardScaler(with_mean=False)
        n = adata_temp.X.shape[0]  # number of rows
        # set dyn scale packet size
        x_len = len(adata_temp.var)
        y_len = len(adata_temp.obs)
        if y_len < 100000:
            dyn_pack = int(x_len / 10)
            pack_size = dyn_pack
        else:
            # 10 pack for every 100,000
            dyn_pack = int((y_len / 100000) * 10)
            pack_size = int(x_len / dyn_pack)
        batch_size = 1000  # pack_size#500  # number of rows in each call to partial_fit
        index = 0  # helper-var
        while index < n:
            partial_size = min(
                batch_size, n - index
            )  # needed because last loop is possibly incomplete
            partial_x = adata_temp.X[index : index + partial_size]
            scaler.partial_fit(partial_x)
            index += partial_size
        adata_temp.X = scaler.transform(adata_temp.X)
    #     else:
    #         sc.pp.scale(adata_temp, zero_center=True, max_value=None, copy=False, layer=None, obsm=None)
    if (train_x_partition != "X") & (train_x_partition in adata_temp.obsm.keys()):
        print("train partition is not in OBSM, defaulting to PCA")
        print("performing highly variable gene selection")
        sc.pp.highly_variable_genes(adata_temp, batch_key=batch_key, subset=False)
        adata_temp = subset_top_hvgs(adata_temp, var_length)
        # Now compute PCA
        sc.pp.pca(
            adata_temp, n_comps=100, use_highly_variable=True, svd_solver="arpack"
        )
        sc.pl.pca_variance_ratio(adata_temp, log=True, n_pcs=100)

        # Batch correction options
        # The script will test later which Harmony values we should use
        if batch_correction == "Harmony":
            print("Commencing harmony")
            adata_temp.obs["lr_batch"] = adata_temp.obs[batch_key]
            batch_var = "lr_batch"
            # Create hm subset
            adata_hm = adata_temp[:]
            # Set harmony variables
            data_mat = np.array(adata_hm.obsm["X_pca"])
            meta_data = adata_hm.obs
            vars_use = [batch_var]
            # Run Harmony
            ho = hm.run_harmony(data_mat, meta_data, vars_use, theta=theta)
            res = (pd.DataFrame(ho.Z_corr)).T
            res.columns = ["X{}".format(i + 1) for i in range(res.shape[1])]
            # Insert coordinates back into object
            adata_hm.obsm["X_pca_back"] = adata_hm.obsm["X_pca"][:]
            adata_hm.obsm["X_pca"] = np.array(res)
            # Run neighbours
            # sc.pp.neighbors(adata_hm, n_neighbors=15, n_pcs=50)
            adata_temp = adata_hm[:]
            del adata_hm
        elif batch_correction == "BBKNN":
            print("Commencing BBKNN")
            sc.external.pp.bbknn(
                adata_temp,
                batch_key=batch_var,
                approx=True,
                metric="angular",
                copy=False,
                n_pcs=50,
                trim=None,
                n_trees=10,
                use_faiss=True,
                set_op_mix_ratio=1.0,
                local_connectivity=15,
            )
        print(
            "adata1 and adata2 are now combined and preprocessed in 'adata' obj - success!"
        )

    # train model
    #    train_x = adata_temp.X
    # train_label = adata_temp.obs[feat_use]
    print("proceeding to train model")
    model = LR_train(
        adata_temp,
        train_x=train_x_partition,
        train_label=feat_use,
        penalty=penalty,
        sparcity=sparcity,
        max_iter=max_iter,
        l1_ratio=l1_ratio,
        tune_hyper_params=tune_hyper_params,
    )
    model.features = list(adata_temp.var.index)
    return model

In [11]:
# projection module
def reference_projection(adata, model, train_x_partition, **kwargs):
    """
    General description.

    Parameters:

    Returns:

    """

    class adata_temp:
        pass

    from sklearn.preprocessing import StandardScaler

    print("Determining model flavour")
    try:
        model_lr = model["Model"]
        print("Consuming celltypist model")
    except:  # hasattr(model, 'coef_'):
        print("Consuming non-celltypist model")
        model_lr = model
    print(model_lr)
    if train_x_partition == "X":
        print("Matching reference genes in the model")
        k_x = np.isin(list(adata.var.index), list(model_lr.features))
        if k_x.sum() == 0:
            raise ValueError(
                f"🛑 No features overlap with the model. Please provide gene symbols"
            )
        print(f"🧬 {k_x.sum()} features used for prediction")
        # slicing adata
        k_x_idx = np.where(k_x)[0]
        # adata_temp = adata[:,k_x_idx]
        adata_temp.var = adata[:, k_x_idx].var
        adata_temp.X = adata[:, k_x_idx].X
        adata_temp.obs = adata[:, k_x_idx].obs
        lr_idx = (
            pd.DataFrame(model_lr.features, columns=["features"])
            .reset_index()
            .set_index("features")
            .loc[list(adata_temp.var.index)]
            .values
        )
        # adata_arr = adata_temp.X[:,list(lr_idexes['index'])]
        # slice and reorder model
        ni, fs, cf = model_lr.n_features_in_, model_lr.features, model_lr.coef_
        model_lr.n_features_in_ = lr_idx.size
        model_lr.features = np.array(model_lr.features)[lr_idx]
        model_lr.coef_ = np.squeeze(
            model_lr.coef_[:, lr_idx]
        )  # model_lr.coef_[:, lr_idx]
        if "partial_scale" in kwargs:
            if partial_scale == True:
                print(
                    "scaling input data, default option is to use incremental learning and fit in mini bulks!"
                )
                # Partial scaling alg
                scaler = StandardScaler(with_mean=False)
                n = adata_temp.X.shape[0]  # number of rows
                # set dyn scale packet size
                x_len = len(adata_temp.var)
                y_len = len(adata.obs)
                if y_len < 100000:
                    dyn_pack = int(x_len / 10)
                    pack_size = dyn_pack
                else:
                    # 10 pack for every 100,000
                    dyn_pack = int((y_len / 100000) * 10)
                    pack_size = int(x_len / dyn_pack)
                batch_size = (
                    1000  # pack_size#500  # number of rows in each call to partial_fit
                )
                index = 0  # helper-var
                while index < n:
                    partial_size = min(
                        batch_size, n - index
                    )  # needed because last loop is possibly incomplete
                    partial_x = adata_temp.X[index : index + partial_size]
                    scaler.partial_fit(partial_x)
                    index += partial_size
                adata_temp.X = scaler.transform(adata_temp.X)
    # model projections
    print("Starting reference projection!")
    if train_x_partition == "X":
        train_x = adata_temp.X
        pred_out = pd.DataFrame(
            model_lr.predict(train_x),
            columns=["predicted"],
            index=list(adata.obs.index),
        )
        proba = pd.DataFrame(
            model_lr.predict_proba(train_x),
            columns=model_lr.classes_,
            index=list(adata.obs.index),
        )
        pred_out = pred_out.join(proba)

    elif train_x_partition in list(adata.obsm.keys()):
        print("{low_dim: this partition modality is still under development!}")
        train_x = adata.obsm[train_x_partition]
        pred_out = pd.DataFrame(
            model_lr.predict(train_x),
            columns=["predicted"],
            index=list(adata.obs.index),
        )
        proba = pd.DataFrame(
            model_lr.predict_proba(train_x),
            columns=model_lr.classes_,
            index=list(adata.obs.index),
        )
        pred_out = pred_out.join(proba)

    else:
        print("{this partition modality is still under development!}")
    ## insert modules for low dim below

    # Simple dynamic confidence calling
    pred_out["confident_calls"] = pred_out["predicted"]
    pred_out.loc[
        pred_out.max(axis=1) < (pred_out.mean(axis=1) + (1 * pred_out.std(axis=1))),
        "confident_calls",
    ] = (
        pred_out.loc[
            pred_out.max(axis=1) < (pred_out.mean(axis=1) + (1 * pred_out.std(axis=1))),
            "confident_calls",
        ].astype(str)
        + "_uncertain"
    )
    # means_ = self.model.scaler.mean_[lr_idx] if self.model.scaler.with_mean else 0
    return (pred_out, train_x, model_lr, adata_temp)

In [5]:
models = {
    "pan_fetal_wget": "https://celltypist.cog.sanger.ac.uk/models/Pan_Fetal_Suo/v2/Pan_Fetal_Human.pkl",
    "YS_wget": "https://storage.googleapis.com/haniffalab/yolk-sac/YS_X_A2_V12_lvl3_ELASTICNET_YS.sav",
}

adatas_dict = {
    "pan_fetal_wget": "https://cellgeni.cog.sanger.ac.uk/developmentcellatlas/fetal-immune/PAN.A01.v01.raw_count.20210429.PFI.embedding.h5ad",
    "YS_wget": "https://app.cellatlas.io/yolk-sac/dataset/23/download",
    "FLIV_wget": "https://app.cellatlas.io/fetal-liver/dataset/1/download",
}

# Variable assignment
train_model = False
feat_use = "cell.labels"
adata_key = "FLIV_wget"  #'fliv_wget_test' # key for dictionary entry containing local or web path to adata/s can be either url or local
data_merge = False  # read and merge multiple adata (useful, but keep false for now)
model_key = "pan_fetal_wget"  #'test_low_dim_ipsc_ys'# key for model of choice can be either url or local
train_x_partition = "X"  # what partition was the data trained on? To keep simple, for now only accepts 'X'
dyn_std = 1.96  # Dynamic cutoffs using std of the mean for each celltype probability, gives a column notifying user of uncertain labels 1 == 68Ci, 1.96 = 95CI
freq_redist = "cell.labels"  #'cell.labels'#'False#'cell.labels'#False # False or key of column in anndata object which contains labels/clusters // not currently implemented
QC_normalise = True  # should data be normalised?

# Read in query data for projection

In [7]:
# setup model parameters:
model_params = {
'penalty':'elasticnet', # can be ["l1","l2","elasticnet"],
'var_length' : 7500,
'batch_key' : 'donor',
'sparcity' : 0.5, #If using LR without optimisation, this controls the sparsity in model C penalty for degree of regularisation
'max_iter' : 1000, #Increase if experiencing max iter issues
'thread_num' : -1,
'l1_ratio' : 0.5, #If using elasticnet without optimisation, this controls the ratio between l1 and l2)
'partial_scale' : False, # should data be scaled in batches?
'tune_hyper_params' : True, # Current implementation is very expensive, intentionally made rigid for now
'batch_correction' : False,
}

if train_model == True:
    from sklearn.preprocessing import StandardScaler
    adata =  scent.
    
    (adatas_dict, data_merge, adata_key, QC_normalise)
    print('adata_loaded')
    import time
    t0 = time.time()
    display_cpu = scent.DisplayCPU()
    display_cpu.start()
    try:
        model_trained = scent.prep_training_data(feat_use = feat_use,
        adata_temp = adata,
        train_x_partition = train_x_partition,
        model_key = model_key + '_lr_model',
        **model_params
        )
        filename =model_name
        pkl.dump(model_trained, open(filename, 'wb'))
    finally: #
        current, peak = display_cpu.stop()
        t1 = time.time()
        time_s = t1-t0
        print('training complete!')
        time.sleep(3)
        print('projection time was ' + str(time_s) + ' seconds')
        print(f"Current memory usage is {current / 10**6}MB; Peak was {peak / 10**6}MB")
        print(f"starting memory usage is" +'' + str(display_cpu.starting))
        print('peak CPU % usage = '+''+ str(display_cpu.peak_cpu))
        print('peak CPU % usage/core = '+''+ str(display_cpu.peak_cpu_per_core))
    model_lr= model_trained
    adata =  scent.load_adatas(adatas_dict, data_merge, adata_key)
else:
    adata =  scent.load_adatas(adatas_dict, data_merge, adata_key,QC_normalise)
    model = scent.load_models(models,model_key)
    model_lr =  model
    
# run with usage logger
import time
t0 = time.time()
display_cpu = scent.DisplayCPU()
display_cpu.start()
try: #code here ##
    pred_out,train_x,model_lr,adata_temp = scent.reference_projection(adata, model_lr, dyn_std,partial_scale,train_x_partition, **model_params)
    if freq_redist != False:
        pred_out['orig_labels'] = adata.obs[freq_redist]
        pred_out = scent.freq_redist_68CI(pred_out,'orig_labels')
        adata.obs['consensus_clus_prediction'] = pred_out['consensus_clus_prediction']
    adata.obs['predicted'] = pred_out['predicted']
    adata_temp.obs = adata.obs
    
    # Estimate top model features for class descrimination
    feature_importance = scent.estimate_important_features(model_lr, 100)
    mat = feature_importance.euler_pow_mat
    top_loadings = feature_importance.to_n_features_long

finally: #
    current, peak = display_cpu.stop()
t1 = time.time()
time_s = t1-t0
print('projection complete!')
time.sleep(3)
print('projection time was ' + str(time_s) + ' seconds')
print(f"Current memory usage is {current / 10**6}MB; Peak was {peak / 10**6}MB")
print(f"starting memory usage is" +'' + str(display_cpu.starting))
print('peak CPU % usage = '+''+ str(display_cpu.peak_cpu))
print('peak CPU % usage/core = '+''+ str(display_cpu.peak_cpu_per_core))

SyntaxError: invalid syntax (3348152279.py, line 40)

In [ ]:
if train_model == True:
    train_label = adata.obs[feat_use].values
else:
    train_label = adata.obs["predicted"].values
scent.report_f1(model_lr, train_x, train_label)